In [0]:
from math import ceil
import requests
from concurrent.futures import ThreadPoolExecutor

In [0]:
bronze_table = 'ws_bees_dev_.db_bronze_dev.tb_openbrewery_api'

domain = 'https://api.openbrewerydb.org/v1/breweries?page='
metadados_url = 'https://api.openbrewerydb.org/v1/breweries/meta'

per_page = 50
all_data = list()

In [0]:
def tt_pages():
    try:
        return ceil(requests.get(metadados_url).json().get('total')/per_page)
    except Exception as e:
        raise(e)

def fetch_data(page):
    response = requests.get(domain+str(page))
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to get page data {page}: {response.status_code}")
        return None

In [0]:
with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(fetch_data, range(1, tt_pages() + 1))

for result in results:
    if result is not None:
        all_data.extend(result)
else:
    openbrewery_new_bronze_df = spark.createDataFrame(all_data).distinct()

In [0]:
try:
    openbrewery_bronze_df = spark.read.table(bronze_table)
    if openbrewery_bronze_df.count() > 0:
        openbrewery_new_bronze_df = openbrewery_new_bronze_df.join(
            openbrewery_bronze_df, on='id', how='left_anti')
except Exception as err:
    print('Is this your first time here?')
finally:
    if openbrewery_new_bronze_df.count() > 0:
        openbrewery_new_bronze_df.write.option("mergeSchema", "true").mode(
            'append').saveAsTable(bronze_table)
        spark.sql("OPTIMIZE "+bronze_table)
    else:
        print("No new records to load.")